<a href="https://colab.research.google.com/github/HaeSeon/NLP-colab/blob/main/KOBERT_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292697 sha256=375753b831224d48144b4b007e1fd46868303530876745510b2fd632cf90a7dd
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.2 MB/s eta 0:00:00


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-erjhg3vh/kobert-tokenizer_f6f7dfedd056470abce0224461efad0c
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-erjhg3vh/kobert-tokenizer_f6f7dfedd056470abce0224461efad0c
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=3dc11fa1842d59f2e6c98b8247c746145ed2e005fe76c43ed474704cb2966e55
  Stored in directory: /tmp/pip-ephem-wheel-cache-shgqxiih/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)


## 데이터

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 경로 설정
import json
path = './drive/MyDrive/stogee-data/train.json'
data_list = []
# JSON 파일 읽기
with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    data_list = data
# 데이터 확인



converted_data = [[item["title"], item["label"]] for item in data_list]
print(converted_data)

[['AI와 빅데이터: 현대 비즈니스에 미치는 영향 분석', 0], ['효율적인 코드 작성을 위한 10가지 꿀팁', 0], ['사이버 보안의 최근 동향과 기업 대응 전략', 0], ['클라우드 마이그레이션: 기업의 성공적인 전략', 0], ['머신러닝으로 예측하는 주식 시장 동향', 0], ['프론트엔드와 백엔드 개발자의 협업 방법', 0], ['최신 웹 개발 프레임워크 비교: React vs. Vue vs. Angular', 0], ['사물인터넷(IoT)이 혁신하는 산업 분야', 0], ['DevOps 문화의 중요성과 구현 방법', 0], ['최근에 등장한 주목받는 프로그래밍 언어 소개', 0], ['소셜 미디어 데이터를 활용한 고객 경험 개선 전략', 0], ['최신 트렌드: 양자 컴퓨팅 기술의 발전과 응용', 0], ['데이터 과학과 비즈니스 인텔리전스의 융합', 0], ['스마트 시티: 도시 인프라를 혁신하는 기술들', 0], ['프로그래머를 위한 새로운 개발 도구 소개', 0], ['코딩 인터뷰에서 빛나는 5가지 알고리즘 문제', 0], ['블록체인 기술의 현재와 미래 전망', 0], ['최신 모바일 앱 개발 트렌드와 도구 소개', 0], ['사용자 데이터 보호를 위한 GDPR 준수 전략', 0], ['퀀텀 컴퓨팅: 미래 컴퓨터의 패러다임 변화', 0], ['IT 프로젝트 관리에서의 성공적인 리더십 전략', 0], ['인공지능과 의료: 진단과 치료에서의 활용', 0], ['소프트웨어 개발에서의 코드 리뷰의 중요성', 0], ['데이터 마이닝과 머신러닝을 활용한 비즈니스 인텔리전스', 0], ['사이버 공격 대응을 위한 효과적인 보안 전략', 0], ['사물인터넷과 스마트 홈: 생활을 더 편리하게 만드는 방법', 0], ['오픈소스 소프트웨어의 발전과 기업의 선택 기준', 0], ['소셜 미디어 플랫폼의 알고리즘과 사용자 경험', 0], ['IT 산업에서의 새로운 비즈니스 모델 탐색', 0], ['디지털 트랜스포메이션과 기업의 경쟁력 강화', 0], [

In [7]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(converted_data, test_size = 0.2, shuffle = True, random_state = 23)

In [8]:
print(len(dataset_train), len(dataset_test))
print(dataset_test)

1008 252
[['전 세계의 다양한 문화와 예술을 느끼는 여행', 1], ['미국 주식 시장에서 성공하는 투자 전략', 2], ['금융 리더와의 토론: 투자의 기술과 성공적인 전략', 2], ['크리에이터의 스케줄 관리와 효율적인 업무', 3], ['IT 비즈니스 애널리스트의 중요한 역할과 책임', 0], ['AI를 활용한 고객 서비스의 향상과 개인화 전략', 0], ['투자 수익을 극대화하는 금융 상품 소개', 2], ['이란의 사막 속 오아시스: 쿠바의 세계 문화 유산 여행', 1], ['투자 성공을 위한 전략적인 자산 배분', 2], ['유튜브 크리에이터의 성장과정 공유', 3], ['동남아시아의 역동적인 거리 음식 탐방 여행', 1], ['창업자를 위한 마인드셋과 자기관리 전략', 4], ['여행자들을 위한 지속 가능한 여행 팁과 명소', 1], ['금융 시장에서의 신규 투자 기회와 도전', 2], ['러시아의 명성 있는 도시 모스크바와 세인트피터부르크 여행', 1], ['인스타그램 크리에이터의 일상 공유', 3], ['클라이언트-서버 아키텍처의 구현 방법', 0], ['스타트업에서의 문화 조성과 팀 빌딩', 4], ['스타트업의 신제품 론칭 전략', 4], ['창업 생태계의 동향과 주요 트렌드', 4], ['창업 아이디어 발굴과 검증 방법', 4], ['남극 대륙에서의 특별한 경험과 모험 여행', 1], ['빅데이터와 실시간 분석을 통한 경영 의사결정', 0], ['세계 각국의 특별한 식도락과 음식문화 여행', 1], ['휴양지에서의 소박한 행복: 시간 가는 줄 모르는 휴식', 1], ['유럽 도시 투어: 걷다 보면 느껴지는 역사와 아름다움', 1], ['AI 기반 네트워크 분석과 보안 대응', 0], ['창업을 위한 적절한 비즈니스 모델 선택', 4], ['스타트업의 효과적인 비즈니스 전략 수립', 4], ['창의성을 끌어올리는 다양한 크리에이터의 히든 스킬', 3], ['IT 프로젝트에서의 스크럼과 애자일 방법론의 적용', 0], ['창업자의 

## 모델 로드

In [9]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [10]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 100
learning_rate = 5e-5

In [26]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [27]:

# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj


In [28]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [29]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)
print(len(data_train), len(data_test))

1008 252


In [30]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [31]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,   # label 클래스 개수 조절
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [32]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [33]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [34]:

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

## 모델 학습

In [35]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/63 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7470190525054932 train acc 0.0625
epoch 1 train acc 0.45634920634920634


  0%|          | 0/16 [00:00<?, ?it/s]

epoch 1 test acc 0.95703125


  0%|          | 0/63 [00:00<?, ?it/s]

...

epoch 9 test acc 1.0


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0017052991315722466 train acc 1.0
epoch 10 train acc 1.0


  0%|          | 0/16 [00:00<?, ?it/s]

epoch 10 test acc 1.0


In [36]:
def predict(predict_sentence): # input = 분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환
    print(another_test)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("IT")
            elif np.argmax(logits) == 1:
                test_eval.append("여행")
            elif np.argmax(logits) == 2:
                test_eval.append("투자")
            else:
                test_eval.append("크리에이터")

        print("해당 글은" + test_eval[0] + "입니다.")

In [38]:
validation_set_data = [
    {"title": "크리에이터의 글로벌 도전기: 세계를 무대로 한 창작 여정", "label": 3},
    {"title": "IT 기술의 최신 동향과 미래 전망", "label": 0},
    {"title": "여행자들을 위한 핫한 여행지 추천", "label": 1},
    {"title": "투자의 기초와 전략: 안전하고 수익성 있는 투자 방법", "label": 2},
    {"title": "창의적인 아이디어로 독자를 매료시키는 크리에이터 비법", "label": 3},
    {"title": "인공지능과 빅데이터의 융합: 현대 비즈니스의 핵심 기술", "label": 0},
    {"title": "세계적인 여행 명소에서 영감 받은 디자인 아이디어", "label": 1},
    {"title": "투자 전문가와 함께하는 자산 운용 전략", "label": 2},
    {"title": "유튜브에서 성공한 크리에이터의 비밀 노하우", "label": 3},
    {"title": "IT 보안의 중요성과 최신 보안 기술 소개", "label": 0},
    {"title": "IT 기술의 혁신과 미래 동향", "label": 0},
    {"title": "여행자들을 위한 신나는 이야기", "label": 1},
    {"title": "스마트한 투자의 전략과 비법", "label": 2},
    {"title": "크리에이터의 세계로 초대합니다", "label": 3},
    {"title": "창업가들의 비밀 일기", "label": 4},
    {"title": "최신 IT 트렌드와 기술 혁신", "label": 0},
    {"title": "힐링 여행의 시작", "label": 1},
    {"title": "투자의 기초와 안전한 전략", "label": 2},
    {"title": "예술적인 창의력의 크리에이터", "label": 3},
    {"title": "창업의 전설과 성공 비밀", "label": 4},
    {"title": "빠르게 변화하는 IT 산업의 현주소", "label": 0},
    {"title": "여행으로 풀어내는 일상의 피로", "label": 1},
    {"title": "투자로 누리는 부의 세계", "label": 2},
    {"title": "창의적인 크리에이터의 일상", "label": 3},
    {"title": "창업의 시작과 성장 비화", "label": 4},
    {"title": "AI와 빅데이터의 미래 전망", "label": 0},
    {"title": "좋은 여행지를 선택하는 노하우", "label": 1},
    {"title": "투자 전문가의 포트폴리오 전략", "label": 2},
    {"title": "유익한 정보를 전달하는 크리에이터", "label": 3},
    {"title": "창업의 고난과 즐거움", "label": 4},
    {"title": "IT 기술의 최신 동향과 발전 가능성", "label": 0},
    {"title": "여행지에서 찾아낸 신비로운 이야기", "label": 1},
    {"title": "투자의 기초와 주식 시장의 분석", "label": 2},
    {"title": "크리에이터의 미래를 여는 창조적 작품", "label": 3},
    {"title": "창업가의 성공 비법과 실패에서의 교훈", "label": 4},
    {"title": "사물인터넷 기술과 IT 산업의 발전", "label": 0},
    {"title": "여행 중 발견한 숨겨진 명소 여행기", "label": 1},
    {"title": "투자의 첫걸음과 안전한 자산 관리", "label": 2},
    {"title": "창의성 넘치는 크리에이터의 일상", "label": 3},
    {"title": "창업의 시작과 성장의 여정", "label": 4},
    {"title": "AI와 머신러닝의 미래 전망과 응용 분야", "label": 0},
    {"title": "로맨틱한 여행으로 빠져드는 이야기", "label": 1},
    {"title": "투자 전문가의 실전 경험과 노하우", "label": 2},
    {"title": "유쾌한 크리에이터와 함께하는 즐거운 시간", "label": 3},
    {"title": "창업가들의 다양한 사업 모델 소개", "label": 4},
    {"title": "AI와 머신러닝의 최신 트렌드와 응용 사례", "label": 0},
    {"title": "아시아 여행의 숨은 명소와 현지 음식 소개", "label": 1},
    {"title": "투자자들을 위한 새로운 금융 상품 소식", "label": 2},
    {"title": "유명 크리에이터들의 창작 과정 비하인드 스토리", "label": 3},
    {"title": "창업자들의 비즈니스 성공을 위한 팁과 전략", "label": 4},
    {"title": "클라우드 컴퓨팅과 빅데이터 분석의 중요성", "label": 0},
    {"title": "여행 중 발견한 특별한 문화 체험 이야기", "label": 1},
    {"title": "투자의 명실공히 통하는 비법과 전략 소개", "label": 2},
    {"title": "크리에이터들의 예술적인 작품 소개와 해설", "label": 3},
    {"title": "창업가들의 성공과 실패에 대한 진솔한 이야기", "label": 4},
    {"title": "사이버 보안과 해킹 방지에 관한 최신 동향", "label": 0},
    {"title": "자연 속의 휴양지 여행과 야외 활동 추천", "label": 1},
    {"title": "스마트 투자자들의 성공적인 자산 운용 전략", "label": 2},
    {"title": "예술과 창의성을 지향하는 크리에이터의 이야기", "label": 3},
    {"title": "창업 시 준비해야 할 핵심 가이드라인 소개", "label": 4},
    {"title": "AI와 머신러닝을 활용한 창의적인 프로젝트 아이디어", "label": 0},
    {"title": "세계 각지의 독특한 문화와 예술 이야기", "label": 1},
    {"title": "금융 전문가들이 추천하는 투자 전략과 비결", "label": 2},
    {"title": "온라인 플랫폼에서 빛나는 새로운 크리에이터의 작품", "label": 3},
    {"title": "창업가들의 도전과 성공 스토리의 진실", "label": 4},
    {"title": "클라우드 보안과 사이버 공격 대응 전략", "label": 0},
    {"title": "세계 각지의 숨겨진 여행지와 이야기", "label": 1},
    {"title": "투자자들을 위한 금융 시장 동향과 전망", "label": 2},
    {"title": "유명 크리에이터들의 창작 브랜딩 비법", "label": 3},
    {"title": "창업가들의 비즈니스 성공과 실패의 기록", "label": 4},
    {"title": "사이버 보안 전문가들의 최신 기술 동향 소개", "label": 0},
    {"title": "여행자들을 위한 독특한 체험과 이야기", "label": 1},
    {"title": "투자 분야에서 주목받는 신진 크리에이터 소개", "label": 2},
    {"title": "다양한 예술 분야에서 활동하는 크리에이터의 일상", "label": 3},
    {"title": "창업가들의 성공 비결과 실패에서 얻은 교훈", "label": 4}
]
validation_set_data = [[item["title"], item["label"]] for item in validation_set_data]

def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for token_ids, valid_length, segment_ids, label in tqdm(dataloader, desc="Evaluating"):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            label = label.long().to(device)

            outputs = model(token_ids, valid_length, segment_ids)
            predicted = torch.argmax(outputs, dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# validation_set_data를 이용하여 BERTDataset 생성
validation_set = BERTDataset(validation_set_data, 0, 1, tokenizer, vocab, max_len, True, False)
# validation_set으로부터 DataLoader 생성
validation_dataloader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=False, num_workers=5)

# evaluate_model 함수 호출
evaluate_model(model, validation_dataloader, device)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Validation Accuracy: 97.33%


In [24]:
# 모델의 가중치와 다른 학습 가능한 매개변수들을 저장
torch.save(model.state_dict(), 'model.pth')
